In [1]:
from datasets import load_dataset

In [3]:
# https://oscar-project.org/post/oscar-2019/
dataset = load_dataset('oscar', 'unshuffled_deduplicated_ko')

Generating train split:   0%|          | 0/3675420 [00:00<?, ? examples/s]

Dataset oscar downloaded and prepared to /home/moonstar/.cache/huggingface/datasets/oscar/unshuffled_deduplicated_ko/1.0.0/84838bd49d2295f62008383b05620571535451d84545037bb94d6f3501651df2. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'text'],
        num_rows: 3675420
    })
})

In [5]:
dataset['train'][0]

{'id': 0,
 'text': 'CIA 프로젝트에서는 데이터베이스로 들어오는 요청을 중간에 수집(Sniffing)하고 수집한 데이터를 분석(Parsing)하여 그로 인한 결과를 판단하여 알릴 수 있는 시스템(Push Service)이 필요하다. 그리고 연구를 위한 가상의 구현 공간(Virtual System)이 필요하다.\n탐지되는 과정에서 오탐가능성을 없애기 위하여 정규표현식을 이용한 결과값 분석이 아닌 Sniffing으로 수집한 Query문으로부터 Table과 Column 명을 추출하기 위한 Query Parser를 구현한다.\nApache 기반의 웹 서버를 구축하고 MySql 기반의 데이터베이스 서버를 구축하였다. 그리고 PHP 기반 웹 사이트를 구현하여 기존에 존재하는 웹 사이트와 유사한 환경을 구현하였다.\n주민등록번호에 대한 부분은 저장하지 않는 방향으로 서버를 구현하였다. 하지만 추후 다양한 내용에 대한 검증을 위해서는 주민등록번호 역시 HASH 암호화된 형태로 저장을 하여 검증 대상에 포함하도록 구현해야 할 것이다.\nPcap 라이브러리를 이용하여 가상 웹사이트에서 항목을 선택할 때 DB 서버에 들어오는 패킷을 Sniffing하여 실행되는 Query 문을 수집하는 기능을 구현하였다. 패킷에서 다양한 정보를 추출 할 수 있지만 그 중에서 Query 문과 IP Address 만이 필요하다.\n하지만 현재 Parser에서 구현되어 있는 입력파일의 형태가 쿼리문 형태의 문자열만을 Parsing하도록 구현이 되어 있어 현재 시연 가능한 수준인 IP 주소를 제외한 쿼리문만 남기도록 소스코드를 변경하였다.\nJava 파일 입출력을 통하여 Log를 Scan한다. Log 확인은 지정한 정규표현식을 이용하여 확인한다. 확인 후 Query Parser를 통해 입력된 Query 문에서 Table 명과 Column 명을 추출한다. 이후 Jsql-Parser를 이용하여 구현한 프로그램으로 각 Token을 생성한다. Token의 조합으로 개인정보를 확인 후 개인정보여부

In [6]:
from tqdm.auto import tqdm

text_data = []
file_count = 0

for sample in tqdm(dataset['train']):
    sample = sample['text'].replace('\n', '')
    text_data.append(sample)
    if len(text_data) == 10_000:
        # once we git the 10K mark, save to file
        with open(f'./Korean/text_{file_count}.txt', 'w', encoding='utf-8') as fp:
            fp.write('\n'.join(text_data))
        text_data = []
        file_count += 1
# after saving in 10K chunks, we will have ~2082 leftover samples, we save those now too
with open(f'./Korean/text_{file_count}.txt', 'w', encoding='utf-8') as fp:
    fp.write('\n'.join(text_data))


  0%|          | 0/3675420 [00:00<?, ?it/s]

In [1]:
from pathlib import Path

paths = [str(x) for x in Path('./Korean').glob('text*.txt')]
paths

['Korean/text_148.txt',
 'Korean/text_118.txt',
 'Korean/text_298.txt',
 'Korean/text_235.txt',
 'Korean/text_356.txt',
 'Korean/text_64.txt',
 'Korean/text_37.txt',
 'Korean/text_53.txt',
 'Korean/text_33.txt',
 'Korean/text_197.txt',
 'Korean/text_264.txt',
 'Korean/text_161.txt',
 'Korean/text_103.txt',
 'Korean/text_71.txt',
 'Korean/text_192.txt',
 'Korean/text_137.txt',
 'Korean/text_16.txt',
 'Korean/text_318.txt',
 'Korean/text_84.txt',
 'Korean/text_209.txt',
 'Korean/text_185.txt',
 'Korean/text_225.txt',
 'Korean/text_164.txt',
 'Korean/text_362.txt',
 'Korean/text_90.txt',
 'Korean/text_223.txt',
 'Korean/text_355.txt',
 'Korean/text_226.txt',
 'Korean/text_81.txt',
 'Korean/text_136.txt',
 'Korean/text_220.txt',
 'Korean/text_257.txt',
 'Korean/text_252.txt',
 'Korean/text_287.txt',
 'Korean/text_4.txt',
 'Korean/text_200.txt',
 'Korean/text_245.txt',
 'Korean/text_360.txt',
 'Korean/text_319.txt',
 'Korean/text_25.txt',
 'Korean/text_66.txt',
 'Korean/text_34.txt',
 'Kore

In [2]:
from tokenizers import ByteLevelBPETokenizer

tokenizer = ByteLevelBPETokenizer()

In [3]:
tokenizer.train(files=paths[:1], vocab_size=30_522, min_frequency=2,
                special_tokens=['<s>', '<pad>', '</s>', '<unk>', '<mask>'])

In [4]:
import os

# os.mkdir('./Korean')
tokenizer.save_model('Korean')

['Korean/vocab.json', 'Korean/merges.txt']